Ce tp a été réalisé à l'aide du standard du NIST relatif à SHA3
(https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.202.pdf)

In [78]:
import numpy as np

n=1088
b=1600
d=256
l=6
w=2^l

In [98]:
def xor(a,b): 
    
    res=''
    
    if (len(a)!=len(b)):
        return ("Error : Strings with differents lengths")
    
    for i in range (len(a)):
        res=str((int(a[-i-1])+int(b[-i-1]))%2)+res
        
    return res

Les deux fonctions ci-dessous ont été utilisé afin de tester la fonction sha-3 sur des exemples du NIST

In [80]:
def h2b(H,n): #convertit une chaine hexadecimal en binaire, n étant le nombre de bits que l'on veut sur le message de sortie de fonction
    
    T=''
    m=len(H)/2
    
    for i in range (m):
        hi=int((H[2*i]+H[2*i+1]),16)
        hi=bin(hi)[2:].zfill(8)
        T+=hi[::-1]
    
    return (T[:n])

In [81]:
def b2h(S): #convertit une chaine binaire en hexadecimal
    
    length=len(S)
    T=S+'0'*((-length)%8)
    
    if (length%8==0):
        m=length/8
    else :
        m=(length//8)+1
    
    H=''
    
    for i in range (m):
        hi=hex(int((T[8*i+4:8*(i+1)])[::-1]+(T[8*i:8*i+4])[::-1],2))[2:]
        
        if (len(hi)==1):
            H+='0'+hi
        else:
            H+=hi
        
    return H[:2*m]

In [82]:
def string_to_matrice(S): #convertit une chaine binaire en matrice tridimensionnelle
    
    a=np.zeros((5,5,w))
    
    for x in range (5):
        for y in range (5):
            for z in range (w):
                a[x][y][z]=int(S[w*(5*y+x)+z])
                
    return a

In [83]:
def Lane(i,j,a):
    
    Lane_out=''
    
    for k in range (w):
        Lane_out+=str(int(a[i][j][k]))
        
    return Lane_out  

In [84]:
def Plane(j,a):
    
    Plane_out=''
    
    for i in range (5):
        Plane_out+=Lane(i,j,a)
        
    return Plane_out

In [85]:
def matrice_to_string(a): #convertit une matrice en chaine binaire
    
    S=''
    
    for j in range (5):
        S+=Plane(j,a)
        
    return S

In [86]:
def theta(a):
    
    a_bis=np.zeros((5,5,w))
    
    #cree une matrice stockant la parite de toutes les colonnes
    C=matrix(GF(2),5,w) 
    for x in range (5):
        for z in range (w):
            for y in range (5):
                C[x,z]+=int(a[x][y][z])
    
    #cree une matrice stockant la somme des parites des deux colonnes adjacentes pour chaque colonne 
    D=matrix(GF(2),5,w) 
    for x in range (5):
        for z in range (w):
            D[x,z]=C[(x-1)%5,z]+C[(x+1)%5,(z-1)%w]
    
    #ajoute la somme des parites des deux colonnes adjacentes pour chaque colonne
    for x in range (5):
        for y in range (5):
            for z in range (w):
                a_bis[x][y][z]=(int(a[x][y][z])+D[x,z])%2
                
    return (a_bis)

In [87]:
def rho(a):
    
    a_bis=np.zeros((5,5,w)) #variable stockant la matrice de sortie de fonction
    
    for z in range (w): #assigne les valeurs de la ligne a[0][0] qui ne subit pas de decalage
        a_bis[0][0][z]=a[0][0][z]
    
    x,y=1,0 #initialise les valeurs de x0,y0
    
    for t in range (24):
        for z in range (w):
            a_bis[x][y][z]=a[x][y][(z-(t+1)*(t+2)/2)%w]
        x,y=y,(2*x+3*y)%5
    
    return a_bis

La matrice utilisée dans la fonction rho est $\left(\begin{array}{ccc}0 & 1\\2 & 3\end{array}\right)$ comme indiqué sur le standard du NIST

In [88]:
def pi(a):
    
    a_bis=np.zeros((5,5,w)) #variable stockant la matrice de sortie de fonction
    
    for x in range (5):
        for y in range (5):
            for z in range (w):
                a_bis[x][y][z]=a[(x+3*y)%5][x][z]
                
    return a_bis

In [89]:
def chi(a):
    
    a_bis=np.zeros((5,5,w)) #variable stockant la matrice de sortie de fonction
    
    for x in range (5):
        for y in range (5):
            for z in range (w):
                a_bis[x][y][z]=(a[x][y][z]+((int(a[(x+1)%5][y][z]+1))&int(a[(x+2)%5][y][z])))%2
    
    return a_bis

In [90]:
def rc(t):
    
    if t%255==0:
        return 1
    
    R='10000000'
    
    for i in range (1,(t%255)+1):
        R='0'+R
        R=xor(R[0],R[8])+R[1:]
        R=R[:4]+xor(R[4],R[8])+R[5:]
        R=R[:5]+xor(R[5],R[8])+R[6:]
        R=R[:6]+xor(R[6],R[8])+R[7:]
        R=R[:8]
        
    return R[0]

In [91]:
def iota(a,ir): #ir correspond au tour de ronde
    
    a_bis=np.zeros((5,5,w)) #variable stockant la matrice de sortie de fonction
    
    for x in range (5):
        for y in range (5):
            for z in range (w):
                a_bis[x][y][z]=a[x][y][z]
    
    RC='0'*w
    
    #la chaine RC generee en fonction de ir ici correspond aux constantes Ci du sujet de tp 
    for j in range (l+1): 
        RC=RC[:2**j-1]+str(rc(j+7*ir))+RC[2**j:]
        
    for z in range (w):
        a_bis[0][0][z]=(a_bis[0][0][z]+int(RC[z]))%2
        
    return a_bis

In [92]:
def Rnd(a,ir):
    return (iota(chi(pi(rho(theta(a)))),ir))

In [93]:
def f(S):
    
    a=string_to_matrice(S)
    
    for ir in range (12+2*l):
        a=Rnd(a,ir)
        
    S_bis=matrice_to_string(a)
    
    return S_bis

In [97]:
def pad(x,m): #renvoie la padding en bit à ajouter au message de taille m pour en faire un message de taille multiple de x
    
    j=(-m-2)%x
    P='1'+'0'*j+'1'
    
    return P

In [95]:
def Sponge(N,d):
    
    P=N+pad(n,len(N))
    k=len(P)/n
    c=b-n
    S='0'*b
    
    for i in range (0,k):
        S=f(xor(S,(P[n*i:n*(i+1)]+('0'*c))))
        
    Z=''
    
    while(len(Z)<d):
        Z+=S[0:n]
        S=f(S)
        
    return Z[0:d]

Test de la fonction sur un message du NIST et son haché ( https://csrc.nist.gov/CSRC/media/Projects/Cryptographic-Standards-and-Guidelines/documents/examples/SHA3-256_1600.pdf )

In [96]:
msg='1100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101110001011100010111000101'
res=b2h(Sponge(msg+'01',d))
res_nist='79F38ADEC5C20307A98EF76E8324AFBFD46CFD81B22E3973C65FA1BD9DE31787'
int(res_nist,16)==int(res,16)

True